# Normalized power coefficient as a function of tip speed ratio

 This is an interactive version of Figure 10 found in Tamaro et al (in review). 

## Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import interpolate
plt.close("all")

 define model constants

In [ ]:
u_inf = np.array([9.5,9.5,9.5])       # free-stream velocity [m/s]
rho   = 1.17                        # air density  [kg/m3]
R     = 65                          # rotor radius [m]

 Import LES data  

In [ ]:
files = ['num_data/tsr8/tsr_8_rigid.mat','num_data/shear01/shear_01_rigid.mat','num_data/shear03/shear_03_rigid.mat']
eta_model = np.load('num_data/salva_eta_modello.npy')
## pre allocate arrays
ct_array    = np.zeros((len(files),4))
eta_p_array = np.zeros((len(files),4))
deta_p_array = np.zeros((len(files),4))
delta_p_modello = np.zeros((len(files),4))
shear_array   = np.zeros(len(files))
tsr_array   = np.zeros(len(files))
gamma_les   = np.linspace(-30,30,7)

 Arrange LES data

In [ ]:
extract_sort = np.array([0,2,3])
c = 0
for filetto in files:
    A = loadmat(filetto)
    cp_les      = A['dati_pow']
    ct_les      = A['dati_ct']
    gamma_array = A['dati_gamma']
    tsr         = A['dati_tsr']
    shear       = A['dati_k']
    eta_modello = eta_model[extract_sort[c],:,:]
    tsr_array[c] = tsr[0]
    shear_array[c] = shear[0]
    cc = 0   
    for i in np.arange(4):
        eta_p_array[c,cc] = ( (cp_les[i*7]) + (cp_les[i*7+6]) ) /2
        deta_p_array[c,cc] = ( (cp_les[i*7+6]) - (cp_les[i*7]) )
        delta_p_modello[c,cc] = eta_modello[i,-1] - eta_modello[i,0]
        ct_array[c,cc] = ( (ct_les[i*7+3]) )/(0.5*rho*np.pi*R**2*u_inf[c]**2)
        cc += 1     
    c += 1         

 Interpolate results on desired ct_target    

In [ ]:
ct_target = np.array([0.4,0.5,0.6,0.7])
idx = np.argsort(shear_array)
tsr_lambda = np.sort(shear_array)
eta_new = np.zeros((len(ct_target),len(shear_array)))
eta_new_modello = np.zeros((len(ct_target),len(shear_array)))
for i in np.arange(len(tsr_array)):
    f = interpolate.interp1d(ct_array[i,:], deta_p_array[i,:],kind='linear',bounds_error=False)#,fill_value = 'extrapolate')
    eta_new[:,i] = f(ct_target)
    
    f = interpolate.interp1d(ct_array[i,:], delta_p_modello[i,:],kind='linear',bounds_error=False)#,fill_value = 'extrapolate')
    eta_new_modello[:,i] = f(ct_target)

 Plot

In [ ]:
n = len(ct_target)
colors = plt.cm.plasma(np.linspace(0.8,0.2,n))
marker_array = np.array(['^', 's', 'o', '*']) 
plt.close("all")
fig, ax = plt.subplots(figsize=(6,3)) 
for i in np.arange(len(ct_target)):
    ax.plot(shear_array,eta_new_modello[i,:],color=colors[i],linewidth=1.75)
    ax.scatter(shear_array,eta_new[i,:],80,color=colors[i],marker=marker_array[i],edgecolors='k',linewidth=0,label=r'$C_T(\gamma=0^\circ)=$' +str(np.round(ct_target[i],2)),zorder=10, clip_on=False)
ax.set_xlabel(r'$k$ [-]')
ax.set_ylabel(r'$\Delta{\eta}_{P,\gamma=\pm30^\circ}$ [-]')
ax.legend(ncol=2,loc='upper left',bbox_to_anchor=(-0.02,1.5),frameon=False,handlelength=1,labelspacing=0.3,columnspacing=0.8,handletextpad=0.1)
ax.set_ylim([0,0.03])
ax.set_yticks(np.linspace(0,0.03,4))
ax.set_xlim([0,0.2])
ax.grid()
plt.subplots_adjust(left=0.2, bottom=0.25, right=0.95, top=0.8, wspace=0.3 , hspace=0.3)